# Lab 3 Exercise 3
## Binary Classification

We are given a dataset that consists of biological features of a person and whether or not they are a smoker.

There are *22 descriptive features* and *1 target column* in the dataset as described in the Table below. You are tasked with creating a predictive model to predict the column "SMK_stat_type_cd", which specifies smoker/non-smoker status of the person. You are expected to use classification methods that was shown in the previous exercises.



|      Column      |                                 Description                                 |
|:----------------:|:---------------------------------------------------------------------------:|
| identified_gender              | male, female                                                                |
| age              | round up to 5 years                                                         |
| height           | round up to 5 cm[cm]                                                        |
| weight           | [kg]                                                                        |
| sight_left       | eyesight(left)                                                              |
| sight_right      | eyesight(right)                                                             |
| hear_left        | hearing left, 1(normal), 2(abnormal)                                        |
| hear_right       | hearing right, 1(normal), 2(abnormal)                                       |
| SBP              | Systolic blood pressure[mmHg]                                               |
| DBP              | Diastolic blood pressure[mmHg]                                              |
| BLDS             | BLDS or FSG(fasting blood glucose)[mg/dL]                                   |
| tot_chole        | total cholesterol[mg/dL]                                                    |
| HDL_chole        | HDL cholesterol[mg/dL]                                                      |
| LDL_chole        | LDL cholesterol[mg/dL]                                                      |
| triglyceride     | triglyceride[mg/dL]                                                         |
| hemoglobin       | hemoglobin[g/dL]                                                            |
| urine_protein    | protein in urine, 1(-), 2(+/-), 3(+1), 4(+2), 5(+3), 6(+4)                  |
| serum_creatinine | serum(blood) creatinine[mg/dL]                                              |
| SGOT_AST         | SGOT(Glutamate-oxaloacetate transaminase) AST(Aspartate transaminase)[IU/L] |
| SGOT_ALT         | ALT(Alanine transaminase)[IU/L]                                             |
| gamma_GTP        | y-glutamyl transpeptidase[IU/L]                                             |
| SMK_stat_type_cd **(Target)** | Smoking state, 0(never), 1(active smoker)                      |


### The stages of this problem can be decomposed as follows:
1. Data Preparation
* Ensure data is in correct format (numerical and not string)
* Normalize the data for better convergence (optional)
* Split the data into train/test subsets

2. Model Selection
* Instanciate models and fit on training data
* Evaluate model performance on testing data
* Select model with best performance

3. Submit model
* Your model will be evaluated on data that is kept separate from training/testing data
* The predictions from your model will be uploaded to the course server where it will be evaluated

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/4c16-labs/code/lab-03/
!unzip -o data.zip
!echo 'data/*' > .gitignore

Mounted at /content/gdrive
/content/gdrive/MyDrive/4c16-labs/code/lab-03
Archive:  data.zip
   creating: data/
  inflating: data/data.csv           
  inflating: data/validation.csv     


In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [25]:
# Data Preparation
# The data is supplied as ".csv" format
# There are 23 columns in csv file, with 22 columns being features and 1 column being the target

# The csv file can be read into as a dataframe using the pandas library
DataFrame = pd.read_csv('data/data.csv')

# Shuffle the data
DataFrame = DataFrame.sample(frac=1)

# Visualize the first 5 rows of the imported dataframe
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
1587,Female,40,155,65,89.0,1.0,1.2,1.0,1.0,130.0,...,60.0,104.0,145.0,13.7,1.0,0.9,11.0,10.0,17.0,0.0
4207,Male,40,175,55,71.0,0.2,0.5,1.0,1.0,120.0,...,81.0,43.0,116.0,16.2,1.0,0.9,19.0,14.0,87.0,1.0
6766,Female,40,165,65,74.1,0.5,0.5,1.0,1.0,110.0,...,75.0,74.0,43.0,13.1,1.0,0.7,20.0,17.0,15.0,1.0
35002,Male,25,165,100,105.0,0.9,1.0,1.0,1.0,160.0,...,41.0,170.0,273.0,16.4,1.0,0.8,33.0,72.0,96.0,1.0
44461,Female,60,155,45,73.3,0.4,0.3,1.0,1.0,111.0,...,49.0,169.0,212.0,11.5,3.0,1.5,28.0,21.0,25.0,1.0
4732,Female,75,145,45,79.0,1.2,0.9,1.0,2.0,130.0,...,49.0,114.0,139.0,14.2,1.0,0.6,34.0,17.0,15.0,0.0


In [26]:
# The column "identified_gender" contains non-numeric data. This cannot be used as is and needs to be converted to numerical representations
# We can encode the identified gender to a numerical representation by letting "female" == 0, and "male" == 1.
# This can be done manually by using conditional statements on the dataframe, but luckily for us there is a simpler method
# We can use the built in LabelEncoder function of sklearn to do this

LabelEncoder = sklearn.preprocessing.LabelEncoder()
LabelEncoder.fit(DataFrame['identified_gender'])
DataFrame['identified_gender'] = LabelEncoder.transform(DataFrame['identified_gender'])

# Label encoder sorts the input column of data in alphabetical order, and then assigns a numerical value to each unique entry
# This results in 'female' being mapped to 0, 'male' being mapped to 1
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
1587,0,40,155,65,89.0,1.0,1.2,1.0,1.0,130.0,...,60.0,104.0,145.0,13.7,1.0,0.9,11.0,10.0,17.0,0.0
4207,1,40,175,55,71.0,0.2,0.5,1.0,1.0,120.0,...,81.0,43.0,116.0,16.2,1.0,0.9,19.0,14.0,87.0,1.0
6766,0,40,165,65,74.1,0.5,0.5,1.0,1.0,110.0,...,75.0,74.0,43.0,13.1,1.0,0.7,20.0,17.0,15.0,1.0
35002,1,25,165,100,105.0,0.9,1.0,1.0,1.0,160.0,...,41.0,170.0,273.0,16.4,1.0,0.8,33.0,72.0,96.0,1.0
44461,0,60,155,45,73.3,0.4,0.3,1.0,1.0,111.0,...,49.0,169.0,212.0,11.5,3.0,1.5,28.0,21.0,25.0,1.0
4732,0,75,145,45,79.0,1.2,0.9,1.0,2.0,130.0,...,49.0,114.0,139.0,14.2,1.0,0.6,34.0,17.0,15.0,0.0


In [27]:
# We should now separate the input features from the target feature and store them as different variables
# We do this by slicing what columns of data we want from the dataframe
# Let's first see the columns available to us by printing DataFrame.columns
print(DataFrame.columns)

Index(['identified_gender', 'age', 'height', 'weight', 'waistline',
       'sight_left', 'sight_right', 'hear_left', 'hear_right', 'SBP', 'DBP',
       'BLDS', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride',
       'hemoglobin', 'urine_protein', 'serum_creatinine', 'SGOT_AST',
       'SGOT_ALT', 'gamma_GTP', 'SMK_stat_type_cd'],
      dtype='object')


In [28]:
# The column titled "SMK_stat_type_cd" is our target, and all remaining variables are our input features.
features_columns = DataFrame.columns[:-1]
target_column = DataFrame.columns[-1:]

DataFrame_X = DataFrame[features_columns]   # Selects only Input features
DataFrame_Y = DataFrame[target_column]      # Selects only Target variable

In [29]:
# Split Data into Train/Test split
# Change the variable "test_frac" to reflect the percentage/fraction of test data in the resulting split
test_frac = 0.15

Train_X, Test_X, Train_Y, Test_Y = train_test_split(DataFrame_X, DataFrame_Y, test_size=test_frac)
print(f"Number of rows for Training:\t{Train_X.shape[0]}\nNumber of rows for Testing:\t{Test_X.shape[0]}")

Number of rows for Training:	42500
Number of rows for Testing:	7500


In [30]:
# EDIT THIS CELL
# Construct a dictionary of prediction models to compare
# Uncomment the below dictionary and insert as many prediction models as you like.
# You may have used binary classification models in previous exercises
# You may also have to import these modules/libraries to be able to use them

# PredictionModels = {
#     'Logistic Regression': code.for.logisitic.regression.model.here(),
#     'K-NN Classificaiton': code.for.KNN.classification.model.here(),
# }

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import normalize

# Define a dictionary of prediction models to compare
PredictionModels = {
    'Logistic Regression': LogisticRegression(penalty='l2', solver='newton-cg'),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5, algorithm="kd_tree"),
    'Support Vector Machine': SGDClassifier(loss='hinge', penalty='l2'),
    'Decision Tree (Depth 5)': DecisionTreeClassifier(max_depth=5),
    'Decision Tree (Depth 4)': DecisionTreeClassifier(max_depth=4)
}

# Normalize the training and testing data
Train_X = normalize(Train_X)
Test_X = normalize(Test_X)

print("Fitting models, please wait...")
for model_name, model in PredictionModels.items():
    print("Fitting model {}".format(name))
    model.fit(Train_X, Train_Y.values.ravel())  # Use values.ravel() to convert to 1D array
    y_pred = model.predict(Test_X)
    print("f1: ", f1_score(Test_Y, y_pred, average='binary'))



    # This loop goes through the models in the variable "PredictionModels"
    # Complete the below code block to fit to the model to training data "Train_X, Train_Y"
    # Peform predictions on the fitted model on the Train set and Test set
    # compute f1 score




Fitting models, please wait...
Fitting model Decision Tree 4
f1:  0.6077348066298343
Fitting model Decision Tree 4
f1:  0.610427226647357
Fitting model Decision Tree 4
f1:  0.5420436989627013
Fitting model Decision Tree 4
f1:  0.7486183218891308
Fitting model Decision Tree 4
f1:  0.7598211434046631


In [31]:
# Select the model with best f1 score and write down the
# corresponding 'key' value from the 'PredictionModels' variable
# Eg if Logistic Regression is chosen then: chosen_model = 'Logistic Regression'
chosen_model = 'Decision Tree (Depth 4)'

In [32]:
# DO NOT EDIT.
# Generate predictions using "chosen_model" and save to file

backend_features = pd.read_csv('data/validation.csv')
backend_features['identified_gender'] = LabelEncoder.transform(backend_features['identified_gender'])
#backend_preds = PredictionModels[chosen_model].predict(backend_features)
backend_preds = PredictionModels[chosen_model].predict(normalize(backend_features[features_columns].to_numpy()))
np.savez_compressed('lab3_ex3_preds', lab3_model=backend_preds)

# Remember to push your changes to the git server for marking!